# <center> Voter Model with Stubborn Agents on Strongly Connected Social Networks

This is the python code used to produce the plots in <i>Voter Model with Stubborn Agents on Strongly Connected Social Networks</i>. We follow the paper organisation. Parameters are set to the one used in the paper but are free to change. Custom functions for our simulations are in `util.py`.

## Setting up
We import the requested modules and set up some parameters for the plots appearance.

In [1]:
# imports
import sys
import numpy as np
from time import time
from random import sample
from scipy.stats import norm
from scipy.linalg import expm
import matplotlib.pyplot as plt
from matplotlib import rc

# our functions for simulations
from util import section4_simu, section5_simu, polarisation

# for confidence intervals
quant = norm().ppf(0.95)

# for plots appearance
color = ["blue", "red", "green", "chocolate", "pink"]
letter = "abcdefghijklmnopqrstuvwxyz"
marker = "x+*o^p"
linestyle = ["-","--","-.",":"]

# latex rendering
rc('font',**{'family':'sans-serif','sans-serif':['Palatino']})
rc('text', usetex=True)

## Section 6, Fig. 1 (left)

Choose the parameters.

In [2]:
n = 1000
N1_init = np.array([250,750])
S = [(100,50), (200,250)]
n_simu = 100
max_time = 40
spacing = 2
seed = None

Simulate using custom function from `util.py`.

In [3]:
length = int(np.floor(max_time/spacing)+1)
N1t = np.zeros((n_simu, N1_init.size, len(S), length))
start = time()

for j,(s0,s1) in enumerate(S):
    for i,n1 in enumerate(N1_init):
        for k in range(n_simu):
            sys.stdout.flush()
            sys.stdout.write("n1={}, s1={}, s0={}. Simu {}/{}. Elapsed time {}\r".format(n1, s1, s0, k+1, n_simu, round(time()-start,1)))
            N1t[k,i,j,:] = section5_simu(n, n1, s1, s0, max_time, spacing, seed)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "D:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-3-279ed69cf2eb>", line 10, in <module>
    N1t[k,i,j,:] = section5_simu(n, n1, s1, s0, max_time, spacing, seed)
  File "D:\PC de Antoine\Documents - data drive\UCL\these\HowOpinionsCrystallise\util.py", line 67, in section5_simu
    u = np.random.choice(range(n))
  File "mtrand.pyx", line 945, in numpy.random.mtrand.RandomState.choice
  File "mtrand.pyx", line 726, in numpy.random.mtrand.RandomState.randint
  File "D:\Anaconda\lib\site-packages\numpy\core\_dtype.py", line 336, in _name_get
    if dtype.isbuiltin == 2:
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "D:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
Attribu

KeyboardInterrupt: 

Compute transition rate Q matrix from system (3) in the paper.

In [ ]:
Q = dict()
for j,(s0,s1) in enumerate(S):
        Q[j] = np.zeros((n-s0-s1+1, n-s0-s1+1))
        for k in range(s1,n-s0+1):
            Q[j][k-s1,k-s1-1] = (k-s1)*(n-k)/(n-1)
            if k<n-s0:
                Q[j][k-s1,k-s1+1] = k*(n-k-s0)/(n-1)
            Q[j][k-s1,k-s1] = - (k-s1)*(n-k)/(n-1) - k*(n-k-s0)/(n-1)

Compute theoretical values for $\mathbb{E}N_1^{(t)}$ using Theorem 2.

In [ ]:
x_axis = np.linspace(0, max_time, length)
expectation = np.zeros((N1_init.size, len(S), length))

for j,(s0,s1) in enumerate(S):
    for c,t in enumerate(x_axis):
        Pt = expm(t*Q[j])
        k_range = np.arange(s1,n-s0+1)
        for i,n1 in enumerate(N1_init):
            expectation[i,j,c] = k_range.dot(Pt[n1-s1,:])

Plot $\mathbb{E}N_1^{(t)}$, theoretical values and empirical averages with confidence intervals.

In [ ]:
elements, names = list(), list()

# plot
for j,(s0,s1) in enumerate(S):

    for i,n1 in enumerate(N1_init):
        mean = np.mean(N1t[:,i,j,:], axis=0)
        std = np.std(N1t[:,i,j,:], axis=0, ddof=1)
        fill_btw = plt.fill_between(x_axis, mean-quant*std/np.sqrt(n_simu), mean+quant*std/np.sqrt(n_simu), alpha=.2, color=color[j])
        plot, = plt.plot(x_axis, expectation[i,j], c=color[j], ls='--', linewidth=.8)
        scatter = plt.scatter(x_axis, mean, c=color[j], marker=marker[j], label="n1 = {}".format(str(n1)))
    elements.append((fill_btw, plot, scatter))
    names.append("$(s_0,s_1) = ({},{})$".format(s0,s1))
    hline = plt.hlines(n*s1/(s0+s1), 0, max_time, color="grey", ls=":")

# limit lines
elements.append(hline)
names.append("$ns_1/(s_0+s_1)$")
        
# save
plt.legend(elements, names, shadow=True, loc="best")
plt.xlabel(r"\textbf{time}")
plt.ylabel(r"\textbf{number of opinion-1 users}")
plt.savefig("section5_n={}_nsimu={}_maxtime={}.pdf".format(n, s1, s0, n_simu, max_time))
plt.show()
plt.close()

## Section 6, Fig. 1 (right)
Choose parameters.

In [ ]:
n = 1000
s0, s1 = 100, 50
Epsilon = np.array([10**(-k) for k in range(2,6)]) # different values of epsilon considered
spacing_theo = .1 # compute total variation every ... time unit

Compute Q from system (3).

In [ ]:
Q = np.zeros((n-s0-s1+1, n-s0-s1+1))
for k in range(s1,n-s0+1):
    Q[k-s1,k-s1-1] = (k-s1)*(n-k)/(n-1)
    if k<n-s0:
        Q[k-s1,k-s1+1] = k*(n-k-s0)/(n-1)
    Q[k-s1,k-s1] = - (k-s1)*(n-k)/(n-1) - k*(n-k-s0)/(n-1)

Compute stationary distribution using [21, section 1.5.7].

In [ ]:
prod = dict()
    
# compute pi_s1
pi_s1 = 0
for k in range(s1+1, n-s0+1):
    prod[k] = 1
    for i in range(s1,k):
        prod[k] *= Q[i-s1,i-s1+1] / Q[i-s1+1,i-s1]
    pi_s1 += prod[k]
pi_s1 = 1/(1+pi_s1)

# create vector of stationary distribution
pi = np.array([pi_s1] + [pi_s1*prod[k] for k in range(s1+1, n-s0+1)])                   
del prod

Compute theoretical mixing time using theorem 4. Total variation is computed every `spacing_theo` time unit.

In [ ]:
mixing_theo = dict()
n1_todo = np.arange(s1, n-s0+1)
mixing_theo = np.zeros((Epsilon.size, n1_todo.size))
t = 0
done = {k: list() for k in range(Epsilon.size)}
done_total = 0

start = time()
while done_total < n1_todo.size*Epsilon.size:
    sys.stdout.flush()
    sys.stdout.write("s1={}, s0={}, t={}, done={}/{}. Elapsed time {}\r"
                     .format(s1, s0, int(t), [len(x) for x in done.values()], n1_todo.size, round(time()-start,3)))
    Pt = expm(t*Q)

    for i,n1 in enumerate(n1_todo):
        totalvar = -1
        for k,eps in enumerate(Epsilon):

            if i not in done[k]:
                if totalvar == -1:
                    totalvar = 0.5 * np.abs(Pt[n1-s1,:]-pi).sum()
                if totalvar < eps:
                    mixing_theo[k,i] = t
                    done[k].append(i)
                    done_total += 1

    t += spacing_theo

Plot mixing times.

In [ ]:
elements, names = list(), list()

for k,eps in enumerate(Epsilon):
    plt.plot(np.arange(s1,n-s0+1), mixing_theo[k,:], 
             color=color[k], linestyle=linestyle[k],
             label=r"$\varepsilon=10^{{-{}}}$".format(k+1))

# show and save
plt.xticks(range(s1,n-s0+1,100))
plt.xlabel(r"\textbf{initial number of opinion-1}")
plt.ylabel(r"\textbf{mixing time}")
plt.legend(shadow=True, loc="lower right")
plt.savefig("section5_mixingtime_n={}_s0={}_s1={}.pdf".format(n,s0,s1))
plt.show()
plt.close()

## Section 7, Fig. 2

Parameters.

In [ ]:
n_nodes = 1000  # total nb users
n_cliques = 2 # nb cliques
clique_size = [500, 500] # cliques sizes
N1 = [100, 400] # n1 in each clique
S = [((250,0),(0,250)), ((250,0),(0,50))] # each element of the list is of the form ((s0 clique C0, s0 clique C1), (s1 clique C0, s1 clique C1))
maxlinks = clique_size[0]*clique_size[1] # max nb links DO NOT CHANGE
n_links = [int(.05*maxlinks), int(.15*maxlinks), int(.75*maxlinks)] # number of links to add
n_cases = len(n_links) # nb of different cases considered
n_simu = 100 # nb simulations
max_time = 20 # max time for simulations
spacing = 1 # register N1 every ... time unit
seed = None # random seed

Simulate using custom function from `util.py`. We re-sample the in-between links at random for each simulation.

In [ ]:
length = int(np.floor(max_time/spacing)+1)
N1t = np.zeros((n_simu, len(S), n_cases, n_cliques, length))
time_start = time()

for r,s in enumerate(S):
    S0, S1 = s[0], s[1]
    for case in range(n_cases):
        for k in range(n_simu):
            sys.stdout.flush()
            sys.stdout.write("Plot {}/{}. Case {}/{}. Simu {}/{}. Elapsed time {}\r".format(r+1, len(S), case+1, n_cases, k+1, n_simu, round(time()-time_start,1)))

            # create graph
            neighbours = {case:dict() for case in range(n_cases)}
            clique = list()

            ### create cliques
            start = 0
            for size in clique_size:
                for i in range(start, start+size):
                    neighbours[i] = np.concatenate((np.arange(start,i), np.arange(i+1,start+size)))
                clique.append(list(range(start, start+size)))
                start += size

            ### interlinks between cliques
            links = n_links[case]
            possible_links = [(i,j) for i in clique[0] for j in clique[1]]
            edges = sample(possible_links, links)
            for l in range(links):
                i, j = edges[l][0], edges[l][1]
                neighbours[i] = np.append(neighbours[i], j)
                neighbours[j] = np.append(neighbours[j], i)
            del edges, possible_links

            # simulate model
            N1t[k,r,case,:,:] = polarisation(neighbours, clique, N1, S1, S0, max_time, spacing, seed)

Compute transition rate Q matrix for each clique using system (3).

In [ ]:
Q = dict()
for r,s in enumerate(S):
    S0, S1 = s[0], s[1]
    for c in range(n_cliques):
        n, n1, s1, s0 = clique_size[c], N1[c], S1[c], S0[c]
        Q[c,r] = np.zeros((n-s0-s1+1, n-s0-s1+1))
        for k in range(s1,n-s0+1):
            Q[c,r][k-s1,k-s1-1] = (k-s1)*(n-k)/(n-1)
            if k<n-s0:
                Q[c,r][k-s1,k-s1+1] = k*(n-k-s0)/(n-1)
            Q[c,r][k-s1,k-s1] = - (k-s1)*(n-k)/(n-1) - k*(n-k-s0)/(n-1)

Compute theoretical values for $\mathbb{E}N_1^{(t)}$ for each clique in the case with no links in between.

In [ ]:
x_axis = np.linspace(0, max_time, length)
expectation = {(c,r): list() for c in range(n_cliques) for r in range(len(S))}

for r,s in enumerate(S):
    S0, S1 = s[0], s[1]
    for c in range(n_cliques):
        n, n1, s1, s0 = clique_size[c], N1[c], S1[c], S0[c]
        for t in x_axis:
            Qexp = expm(t*Q[c,r])
            k_range = np.arange(S1[c], n-s0+1)
            expectation[c,r].append(k_range.dot(Qexp[n1-s1, :]))

Plot.

In [ ]:
names_theo = ["$C_0$", "$C_1$"]
linestyle = ["--","-."]
fig, ax = plt.subplots(1,2, figsize=(12,4))
percents = [5,15,75]

for r,s in enumerate(S):
    elements, names = list(), list()
    S0, S1 = s[0], s[1]
    elements, names = list(), list()
    
    for c in range(n_cliques):
        plot, = ax[r].plot(x_axis, expectation[c,r], c="black", ls=linestyle[c], linewidth=.8)
        elements.append(plot)
        names.append("no links ("+names_theo[c]+")")
        
        # add C0 and C1 points
        if (r,c)==(0,1):
            ax[r].text(0, expectation[c,r][0]-40, names_theo[c])
        else:
            ax[r].text(0, expectation[c,r][0]+20, names_theo[c])

    for l in range(n_cases):
        for c in range(n_cliques):
            mean = np.mean(N1t[:,r,l,c,:], axis=0)
            std = np.std(N1t[:,r,l,c,:], axis=0, ddof=1)
            fill_btw = ax[r].fill_between(x_axis, mean-quant*std/np.sqrt(n_simu), mean+quant*std/np.sqrt(n_simu), alpha=.2, color=color[l])
            scatter = ax[r].scatter(x_axis, mean, c=color[l], marker=marker[c])
            elements.append((fill_btw, scatter))
            names.append(r"${}$\% links (".format(percents[l], letter[c]) + names_theo[c] +")")
        

# show and save
ax[1].legend(elements, names, loc=[1.1,0.2], shadow=True)
ax[0].set_xlabel(r"\textbf{time}")
ax[1].set_xlabel(r"\textbf{time}")
ax[0].set_ylabel(r"\textbf{number of opinion-1 users}")

plt.tight_layout(pad=2.5)
plt.savefig("polarisation_n={}_cliquesizes={}-{}_N1={}-{}_nsimus={}.pdf"
            .format(n_nodes, clique_size[0], clique_size[1], N1[0], N1[1], n_simu))
plt.show()
plt.close()